# THIS NOTEBOOK IS STILL IN PROGRESS

In [5]:
! pip install -U milvus "farm-haystack[crawler]" milvus-haystack 

In [6]:
from milvus import default_server
default_server.start()

In [7]:
from haystack import Pipeline
from haystack.nodes import Crawler, PreProcessor, EmbeddingRetriever
from milvus_haystack import MilvusDocumentStore

document_store = MilvusDocumentStore(recreate_index=True, return_embedding=True, similarity="cosine")
crawler = Crawler(urls=["https://milvus.io/docs/"], crawler_depth=1, overwrite_existing_files=True, output_dir="crawled_files")
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=False,
    clean_header_footer=True,
    split_by="word",
    split_length=500,
    split_respect_sentence_boundary=True,
)
retriever = EmbeddingRetriever(document_store=document_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1")

indexing_pipeline = Pipeline()
indexing_pipeline.add_node(component=crawler, name="crawler", inputs=['File'])
indexing_pipeline.add_node(component=preprocessor, name="preprocessor", inputs=['crawler'])
indexing_pipeline.add_node(component=retriever, name="retriever", inputs=['preprocessor'])
indexing_pipeline.add_node(component=document_store, name="document_store", inputs=['retriever'])

indexing_pipeline.run()

RPC error: [drop_collection], <MilvusException: (code=1, message=InvalidateCollectionMetaCache failed, proxyID = 1, err = stack trace: /Users/jibin/Working/milvus-lite/milvus_binary/milvus/pkg/tracer/stack_trace.go:51 github.com/milvus-io/milvus/pkg/tracer.StackTrace
/Users/jibin/Working/milvus-lite/milvus_binary/milvus/internal/util/grpcclient/client.go:515 github.com/milvus-io/milvus/internal/util/grpcclient.(*ClientBase[...]).Call
/Users/jibin/Working/milvus-lite/milvus_binary/milvus/internal/util/grpcclient/client.go:529 github.com/milvus-io/milvus/internal/util/grpcclient.(*ClientBase[...]).ReCall
/Users/jibin/Working/milvus-lite/milvus_binary/milvus/internal/distributed/proxy/client/client.go:76 github.com/milvus-io/milvus/internal/distributed/proxy/client.wrapGrpcCall[...]
/Users/jibin/Working/milvus-lite/milvus_binary/milvus/internal/distributed/proxy/client/client.go:109 github.com/milvus-io/milvus/internal/distributed/proxy/client.(*Client).InvalidateCollectionMetaCache
/User

MilvusException: <MilvusException: (code=1, message=InvalidateCollectionMetaCache failed, proxyID = 1, err = stack trace: /Users/jibin/Working/milvus-lite/milvus_binary/milvus/pkg/tracer/stack_trace.go:51 github.com/milvus-io/milvus/pkg/tracer.StackTrace
/Users/jibin/Working/milvus-lite/milvus_binary/milvus/internal/util/grpcclient/client.go:515 github.com/milvus-io/milvus/internal/util/grpcclient.(*ClientBase[...]).Call
/Users/jibin/Working/milvus-lite/milvus_binary/milvus/internal/util/grpcclient/client.go:529 github.com/milvus-io/milvus/internal/util/grpcclient.(*ClientBase[...]).ReCall
/Users/jibin/Working/milvus-lite/milvus_binary/milvus/internal/distributed/proxy/client/client.go:76 github.com/milvus-io/milvus/internal/distributed/proxy/client.wrapGrpcCall[...]
/Users/jibin/Working/milvus-lite/milvus_binary/milvus/internal/distributed/proxy/client/client.go:109 github.com/milvus-io/milvus/internal/distributed/proxy/client.(*Client).InvalidateCollectionMetaCache
/Users/jibin/Worki

In [1]:
import os
from dotenv import load_dotenv
from haystack import Pipeline
from haystack.nodes import EmbeddingRetriever, PromptNode, PromptTemplate, AnswerParser
from milvus_haystack import MilvusDocumentStore

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

document_store = MilvusDocumentStore()

retriever = EmbeddingRetriever(document_store=document_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1")
template = PromptTemplate(prompt="deepset/question-answering", output_parser=AnswerParser())
prompt_node = PromptNode(model_name_or_path="gpt-4", default_prompt_template=template, api_key=OPENAI_API_KEY, max_length=200)

query_pipeline = Pipeline()
query_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
query_pipeline.add_node(component=prompt_node, name="PromptNode", inputs=["Retriever"])

Failed to create new connection using: 356c5cc01259481f88f9983f21a7839c


MilvusException: <MilvusException: (code=2, message=Fail connecting to server on localhost:19530. Timeout)>

In [9]:
default_server.stop()